In [ ]:
import netCDF4 as nc
import cartopy.crs as ccrs
from datetime import date, timedelta, datetime
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import dask.array as da
import os
import cftime

############ INPUT PATH ##############

rootpath_in = '/bdd/CORDEX/output/'
CORDEX_domain = 'EUR-11'
GCM = 'CNRM-CERFACS-CNRM-CM5'
scenario = 'rcp45'
RCM = 'CNRM-ALADIN63'
em = 'r1i1p1' #ensemble_member
ver='v2' #version
freq='3hr' #frequency
ver_2='latest'
#version_3=..
#version_4=..
HH_1s='0130' #start hour averaged variables (see README.txt)
HH_2s='0300' #start hour instantaneous variables 
HH_1e='2230' #end hour averaged variables 
HH_2e='0000' #end hour instantaneous variables

forcing_in = rootpath_in+CORDEX_domain+'/CNRM/'+GCM+'/'+scenario+'/'+em+'/'+RCM+'/'+ver+'/'+freq 

############################# TIME RANGE DEFINITION ###########

first_year = 2006
last_year = 2100

years=np.arange(first_year, last_year+1, 1) #right value excluded

################################# MASK CALL ######

mask_ocean=xr.open_dataset('/modfs/project/input4CORDEX/output/EUR-11/grid_lambertian.nc') 
mask=mask_ocean.LANDMASK.data

#################### VARIABLES CALL ##################

for a in years: #y is already one of the dimension
    rsds_open= forcing_in+'/rsds/'+ver_2+'/rsds_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_1s+'-'+str(a)+'1231'+HH_1e+'.nc'
    rlds_open= forcing_in+'/rlds/'+ver_2+'/rlds_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_1s+'-'+str(a)+'1231'+HH_1e+'.nc'
    sfcWind_open= forcing_in+'/sfcWind/'+ver_2+'/sfcWind_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_2s+'-'+str(a+1)+'0101'+HH_2e+'.nc'
    tas_open= forcing_in+'/tas/'+ver_2+'/tas_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_2s+'-'+str(a+1)+'0101'+HH_2e+'.nc'
    huss_open= forcing_in+'/huss/'+ver_2+'/huss_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_2s+'-'+str(a+1)+'0101'+HH_2e+'.nc'
    pr_open= forcing_in+'/pr/'+ver_2+'/pr_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_1s+'-'+str(a)+'1231'+HH_1e+'.nc'
    ps_open= forcing_in+'/ps/'+ver_2+'/ps_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'0101'+HH_2s+'-'+str(a+1)+'0101'+HH_2e+'.nc'
    
    shortwave_radiation=xr.open_mfdataset(rsds_open, combine='by_coords', parallel=True,  chunks={'time': 200}) #chuncks, to optimize memory use
    longwave_radiation=xr.open_mfdataset(rlds_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    surface_wind=xr.open_mfdataset(sfcWind_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    air_temperature=xr.open_mfdataset(tas_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    specific_humidity=xr.open_mfdataset(huss_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    total_precipitation=xr.open_mfdataset(pr_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    surface_pressure=xr.open_mfdataset(ps_open, combine='by_coords', parallel=True,  chunks={'time': 200})
    
    times= shortwave_radiation.time
    lat = shortwave_radiation.lat
    lon = shortwave_radiation.lon

    Fill_value = 9.96921e+36
    SWdown=shortwave_radiation.rename({'rsds': 'SWdown'}).SWdown*mask
    
    LWdown=longwave_radiation.rename({'rlds': 'LWdown'}).LWdown*mask

    Wind=surface_wind.rename({'sfcWind': 'Wind'}).Wind*mask
   
    Tair=air_temperature.rename({'tas': 'Tair'}).Tair*mask
    
    Qair=specific_humidity.rename({'huss': 'Qair'}).Qair*mask

    PSurf = surface_pressure.rename({'ps': 'PSurf'}).PSurf*mask

    Prec = total_precipitation.rename({'pr': 'Prec'}).Prec*mask
    
    tstep_data = np.arange(0, len(PSurf.time))
    first_value = 3600 + 1800 # the original files start at 01:30
    time_step = 3*3600 # 3hourly data
    times_data= first_value +((tstep_data) * time_step)

    x = shortwave_radiation.x
    y = shortwave_radiation.y
    
    Rainf = np.zeros((len(PSurf.time), len(Prec.y), len(Prec.x)))
    Rainf=np.where(Tair > 273.15, Prec, 0)

    Snowf = np.zeros((len(PSurf.time), len(Prec.y), len(Prec.x))) 
    Snowf=np.where(Tair <= 273.15, Prec, 0)

############## OUTPUT PATH #########################################

        
    rootpath_out = '/modfs/project/input4CORDEX/output/' 

    forcing_out = rootpath_out+CORDEX_domain+'/CNRM/'+GCM+'/'+scenario+'/'+em+'/'+RCM+'/'+ver+'/'+freq+'/forcing_'+CORDEX_domain+'_'+GCM+'_'+scenario+'_'+em+'_'+RCM+'_'+ver+'_'+freq+'_'+str(a)+'.nc'

    if not os.path.isdir(os.path.dirname(forcing_out)):
        os.makedirs(os.path.dirname(forcing_out))

########### CREATION OUTPUT DATASET ################################### 

    ds = nc.Dataset(forcing_out, 'w')

    ds.createDimension('tstep', None)
    ds.createDimension('x', len(x))
    ds.createDimension('y', len(y))

    Lat = ds.createVariable('lat', 'f4', ('y', 'x'))
    Lon = ds.createVariable('lon', 'f4', ('y', 'x'))
    Lat[:]=lat.data
    Lon[:]=lon.data

    time = ds.createVariable('time', 'f8', ('tstep',))
    time[:] = times_data

    xproj = ds.createVariable('x_proj','f4', ('x'))
    yproj = ds.createVariable('y_proj','f4', ('y'))
    xproj[:] = x.data#[100:180]
    yproj[:] = y.data#[100:255]
    rp = ds.createVariable('Lambert_Conformal', 'i')

    psurf = ds.createVariable('PSurf', 'f4', ('tstep', 'y', 'x'), fill_value=Fill_value)
    psurf[:,:,:] = PSurf.data
    tair = ds.createVariable('Tair', 'f4', ('tstep',  'y', 'x'), fill_value=Fill_value)
    tair[:,:,:] = Tair.data
    swdown = ds.createVariable('SWdown', 'f4', ('tstep',  'y', 'x'), fill_value=Fill_value)
    swdown[:,:,:] = SWdown.data
    lwdown = ds.createVariable('LWdown', 'f4', ('tstep',  'y', 'x'), fill_value=Fill_value)
    lwdown[:,:,:] = LWdown.data
    qair = ds.createVariable('Qair', 'f4', ('tstep', 'y', 'x'), fill_value=Fill_value)
    qair[:,:,:] = Qair.data
    wind = ds.createVariable('Wind', 'f4', ('tstep', 'y', 'x'), fill_value=Fill_value)
    wind[:,:,:] = Wind.data
    snowf = ds.createVariable('Snowf', 'f4', ('tstep',  'y', 'x'), fill_value=Fill_value)
    snowf[:,:,:] = Snowf
    rainf = ds.createVariable('Rainf', 'f4', ('tstep',  'y', 'x'), fill_value=Fill_value)
    rainf[:,:,:] = Rainf
    contfrac= ds.createVariable('Contfrac', 'f4',  ('y', 'x'), fill_value=Fill_value)
    contfrac[:,:] = mask

    variables_attrs = {
        'time':{'title': 'Time', 'long_name': 'Time axis', 'axis': 'T', 'time_origin': f'{a}-JAN-01 00:0:00', 'units': f'seconds since {a}-01-01 00:00:00',
               'calendar': 'standard'},
        'lat': {'standard_name':'latitude', 'long_name' : 'Latitude', 'units' : 'degrees_north'},
        'lon': {'standard_name':'longitude', 'long_name' : 'Longitude', 'units' : 'degrees_east'},
        'x_proj': {'standard_name':'projection_x_coordinate', 'long_name' : 'x coordinate of projection', 'units' : 'km', 'axis': 'X'},
        'y_proj': {'standard_name':'projection_y_coordinate', 'long_name' : 'y coordinate of projection', 'units' : 'km', 'axis': 'Y'},
        'Lambert_Conformal':{'grid_mapping_name' : 'lambert_conformal_conic', 
                             'latitude_of_projection_origin': 49.5, 'standard_parallel': -49.5, 'longitude_of_central_meridian': 10.5},
        'SWdown': {'units': 'W m-2', 'long_name': 'Surface Downwelling Shortwave Radiation', 'cell_methods': 'time: mean(centre)',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'LWdown': {'units': 'W m-2', 'long_name': 'Surface Downwelling Longwave Radiation', 'cell_methods': 'time: mean(centre)',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Tair':   {'units': 'K', 'long_name': 'Near-surface Air Temperature', 'cell_methods': 'time: instantaneous',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Qair':   {'units': 'kg kg-1', 'long_name': 'Near-surface specific humidity', 'cell_methods': 'time: instantaneous',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Wind':   {'units': 'm s-1', 'long_name': 'Near-surface Wind Speed', 'cell_methods': 'time: instantaneous',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'PSurf':  {'units': 'Pa', 'long_name': 'Surface Air Pressure', 'cell_methods': 'time: instantaneous',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Rainf':  {'units': 'kg m-2 s-1', 'long_name': 'Rainfall Flux', 'cell_methods': 'time: mean(centre)',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Snowf':  {'units': 'kg m-2 s-1', 'long_name': 'Snowfall Flux', 'cell_methods': 'time: mean(centre)',
                  'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'},
        'Contfrac': {'long_name': 'Land Area Fraction', 'grid_mapping' : 'Lambert_Conformal', 'coordinates' : 'lat lon'}
    }
    
    for var, attrs in variables_attrs.items():
        for attr_name, attr_value in attrs.items():
            ds.variables[var].setncattr(attr_name, attr_value)

    ds.close() 
